# Задание по программированию: Выбор семейства распределений в наивном байесе

Курс "Обучение на размеченных данных" https://www.coursera.org/learn/supervised-learning/home/welcome

Автор: Сметанин Александр

Дата: 07.06.2018

Вам предлагается выяснить, какое распределение лучше использовать в наивном байесовском классификаторе в зависимости от вида признаков.

Загрузите датасеты digits и breast_cancer из sklearn.datasets. Выведите несколько строчек из обучающих выборок и посмотрите на признаки. С помощью sklearn.cross_validation.cross_val_score c настройками по умолчанию и вызова метода mean() у возвращаемого этой функцией numpy.ndarray, сравните качество работы наивных байесовских классификаторов на этих двух датасетах. Для сравнения предлагается использовать BernoulliNB, MultinomialNB и GaussianNB. Насколько полученные результаты согласуются с рекомендациями из лекций?

In [1]:
from sklearn.datasets import load_digits, load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Load datasets
ds_digits = load_digits()
ds_cancer = load_breast_cancer()
print ds_digits.DESCR

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [3]:
print ds_cancer.DESCR

Breast Cancer Wisconsin (Diagnostic) Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.

        

In [4]:
ds_digits.data[:5]

array([[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
        15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
        12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
         0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
        10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,
         9.,  0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7.,
        15., 16., 16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,
         0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16.,
        16.,  6.,  0.,  0.,  0.,  0.,  0., 11., 16., 10.,  0.,  0.],
       [ 0.,  0.,  0.,  4., 15., 12.,  0.,  0.,  0.,  0.,  3., 16., 15.,
        14.,  0.,  0.,  0.,  0.,  8., 13.,  8., 16.,  0.,  0.,  0.,  0.,
         1.,  6., 15., 11.,  0.,  0.,  0.,  1.,  8., 13., 15.,  1.,  0.,
         0.,  0.,  9., 16., 16.,  5.,  0.,  0.,  0.,  0.,  

In [5]:
ds_cancer.data[:5]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [6]:
# Cross validation scores
digits_CV_score_bern = cross_val_score(BernoulliNB(), ds_digits.data, ds_digits.target)
digits_CV_score_multi = cross_val_score(MultinomialNB(), ds_digits.data, ds_digits.target)
digits_CV_score_gauss = cross_val_score(GaussianNB(), ds_digits.data, ds_digits.target)
cancer_CV_score_bern = cross_val_score(BernoulliNB(), ds_cancer.data, ds_cancer.target)
cancer_CV_score_multi = cross_val_score(MultinomialNB(), ds_cancer.data, ds_cancer.target)
cancer_CV_score_gauss = cross_val_score(GaussianNB(), ds_cancer.data, ds_cancer.target)

In [7]:
# Show the results
meths = ["Бернулли", "Мультиномиальный", "Гаусса"]*2
dats = ["Digits"]*3 + ["Breast_Cancer"]*3
vals = [digits_CV_score_bern.mean(),
       digits_CV_score_multi.mean(),
       digits_CV_score_gauss.mean(),
       cancer_CV_score_bern.mean(),
       cancer_CV_score_multi.mean(),
       cancer_CV_score_gauss.mean()]
for meth, dat, val in zip(meths, dats, vals):
    print 'Средняя точность метода {} на наборе {} = {}'.format(meth, dat, val)

Средняя точность метода Бернулли на наборе Digits = 0.825823650778
Средняя точность метода Мультиномиальный на наборе Digits = 0.870877148974
Средняя точность метода Гаусса на наборе Digits = 0.818600380355
Средняя точность метода Бернулли на наборе Breast_Cancer = 0.627420402859
Средняя точность метода Мультиномиальный на наборе Breast_Cancer = 0.894579040193
Средняя точность метода Гаусса на наборе Breast_Cancer = 0.936749280609


In [8]:
# This function writes into a file
def write_answer(filename, value):
    with open(filename, "w") as fout:
        fout.write(str(value))

## Вопрос 1

Каким получилось максимальное качество классификации на датасете breast_cancer?

In [9]:
print 'Масимальная точность на наборе breast_cancer равна {:.3f}, рассчитанно методом {}.'\
    .format(cancer_CV_score_gauss.mean(), 'Гаусса')
write_answer('Week_5_Task_2_Answer_1.txt', cancer_CV_score_gauss.mean())

Масимальная точность на наборе breast_cancer равна 0.937, рассчитанно методом Гаусса.


## Вопрос 2

Каким получилось максимальное качество классификации на датасете digits?

In [10]:
print 'Масимальная точность на наборе digits равна {:.3f}, рассчитанно методом {}.'\
    .format(digits_CV_score_multi.mean(), 'Мультиномиальный')
write_answer('Week_5_Task_2_Answer_2.txt', digits_CV_score_multi.mean())

Масимальная точность на наборе digits равна 0.871, рассчитанно методом Мультиномиальный.


## Вопрос 3

Выберите верные утверждения и запишите их номера через пробел (в порядке возрастания номера):

1) На вещественных признаках лучше всего сработал наивный байесовский классификатор с распределением Бернулли

2) На вещественных признаках лучше всего сработал наивный байесовский классификатор с мультиномиальным распределением

3) Мультиномиальное распределение лучше показало себя на выборке с целыми неотрицательными значениями признаков

4) На вещественных признаках лучше всего сработало нормальное распределение

In [11]:
write_answer('Week_5_Task_2_Answer_3.txt', '3 4')